In [1]:
import torch
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np

In [2]:
from aurora import Aurora

model = Aurora()
# model.load_checkpoint("microsoft/aurora", "aurora-0.25-finetuned.ckpt")
model.load_checkpoint_local(
    "/Users/pmpaquet/Projects/Stanford/CS229S/Project/Project_aurora/models/hf_ckpt/aurora-0.25-finetuned.ckpt"
)

/Users/pmpaquet/Projects/Stanford/CS229S/Project/Project_aurora/aurora_229s/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
def compress_param(name, param):
    if len(param.shape) <= 1:
        return False
    elif 'lora' in name:
        return False
    elif not (name.startswith('backbone.encoder_layers') or name.startswith('backbone.decoder_layers')):
        return False
    else:
        return True

In [20]:
# Load the model
for name,param in model.named_parameters():
    if compress_param(name, param) or True:
        print(f"{name:64} -- {param.shape}")

encoder.atmos_latents                                            -- torch.Size([3, 512])
encoder.surf_level_encoding                                      -- torch.Size([512])
encoder.surf_mlp.net.0.weight                                    -- torch.Size([2048, 512])
encoder.surf_mlp.net.0.bias                                      -- torch.Size([2048])
encoder.surf_mlp.net.2.weight                                    -- torch.Size([512, 2048])
encoder.surf_mlp.net.2.bias                                      -- torch.Size([512])
encoder.surf_norm.weight                                         -- torch.Size([512])
encoder.surf_norm.bias                                           -- torch.Size([512])
encoder.pos_embed.weight                                         -- torch.Size([512, 512])
encoder.pos_embed.bias                                           -- torch.Size([512])
encoder.scale_embed.weight                                       -- torch.Size([512, 512])
encoder.scale_embed.bias    

# Get total size of the model

In [38]:
total_cnt = 0
compress_cnt = 0
noncomp_cnt = 0
for name,param in model.named_parameters():
    n_params = np.prod(list(param.shape))
    total_cnt += n_params
    if compress_param(name, param):
        compress_cnt += n_params
    else:
        noncomp_cnt += n_params


print(f"Total number of Parameters: {total_cnt}")
print(f"Number of compressible arameters: {compress_cnt}")
print(f"Number of uncompressible arameters: {noncomp_cnt}")
print("Compression ratio to size reductions:")
for ratio in [0.75, 0.5, 0.25]:
    print(f"{ratio} -- {100.0*(noncomp_cnt + (ratio * compress_cnt)) / total_cnt:6.2f}%")

Total number of Parameters: 1259150992
Number of compressible arameters: 1238630400
Number of uncompressible arameters: 20520592
Compression ratio to size reductions:
0.75 --  75.41%
0.5 --  50.81%
0.25 --  26.22%


# Figure out how to use SVD to decompose data

In [50]:
model = Aurora()
# model.load_checkpoint("microsoft/aurora", "aurora-0.25-finetuned.ckpt")
model.load_checkpoint_local(
    "/Users/pmpaquet/Projects/Stanford/CS229S/Project/Project_aurora/models/hf_ckpt/aurora-0.25-finetuned.ckpt"
)

for name,param in model.named_parameters():
    if compress_param(name, param):
        break

In [51]:
U, S, Vh = torch.linalg.svd(param.detach(), full_matrices=False)
print(name, param.shape, U.shape, S.shape, Vh.shape)

backbone.encoder_layers.0.blocks.0.norm1.ln_modulation.1.weight torch.Size([1024, 512]) torch.Size([1024, 512]) torch.Size([512]) torch.Size([512, 512])


In [52]:
print(param.shape)
print((U[:,:64] @ torch.diag(S[:64]) @ Vh[:64]).shape)

torch.Size([1024, 512])
torch.Size([1024, 512])


In [42]:
ratio = 0.125 # greatest reduction
k = round(S.shape[0] * ratio)
print(ratio, k, U[:,:k].shape, S[:k].shape, Vh[:k,:].shape)

0.125 64 torch.Size([1024, 64]) torch.Size([64]) torch.Size([64, 512])


### See how to update the parameter values

In [46]:
model = torch.nn.Sequential(
    torch.nn.Linear(5, 1)
)

with torch.no_grad():
    for name,param in model.named_parameters():
        print(name, param.shape, param)
        param += 1.0

print()

for name,param in model.named_parameters():
    print(name, param.shape, param)

0.weight torch.Size([1, 5]) Parameter containing:
tensor([[-0.1483, -0.4368, -0.1186,  0.3334, -0.2606]], requires_grad=True)
0.bias torch.Size([1]) Parameter containing:
tensor([0.1484], requires_grad=True)

0.weight torch.Size([1, 5]) Parameter containing:
tensor([[0.8517, 0.5632, 0.8814, 1.3334, 0.7394]], requires_grad=True)
0.bias torch.Size([1]) Parameter containing:
tensor([1.1484], requires_grad=True)


In [48]:
torch.diag(torch.tensor([1, 2, 3]))

tensor([[1, 0, 0],
        [0, 2, 0],
        [0, 0, 3]])

# Compression loop

In [55]:
from pathlib import Path
save_dir = Path('/Users/pmpaquet/Projects/Stanford/CS229S/Project/Project_aurora/models/svd_models')

for ratio in [0.125, 0.25, 0.5, 0.75]:
    print("Ratio:", ratio)

    model = Aurora()
    model.load_checkpoint_local(
        "/Users/pmpaquet/Projects/Stanford/CS229S/Project/Project_aurora/models/hf_ckpt/aurora-0.25-finetuned.ckpt"
    )

    # Update
    with torch.no_grad():
        for name,param in model.named_parameters():
            if compress_param(name, param):
                U, S, Vh = torch.linalg.svd(param.detach(), full_matrices=False)
                k = round(S.shape[0] * ratio)
                
                # Apply compression
                param = U[:,:k] @ torch.diag(S[:k]) @ Vh[:k]


    torch.save(model.state_dict(), str(save_dir / f'aurora-0.25-finetuned-svd-{ratio}.ckpt'))

Ratio: 0.125
Ratio: 0.25
Ratio: 0.5
Ratio: 0.75


In [54]:
model = Aurora()
model.load_checkpoint_local(str(save_dir / f'aurora-0.25-finetuned-svd-{ratio}.ckpt'))

In [ ]:
# if this doesn't work upon inference, I can always modify the 